In [21]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tqdm import trange

import actor_critic
import importlib
importlib.reload(actor_critic)
from actor_critic import ActorCritic, EnvBatch

In [22]:
environ_name = "CartPole-v0"

env = EnvBatch( environ_name, n_envs=10 )
ac = ActorCritic(
                 env=env,
                 entropy_coef=0.01,
                 scale_model_inputs=True,
                 actor_learning_rate=0.001,
                 critic_learning_rate=0.001
                )

/Users/chris/anaconda3/envs/python37/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1503: RuntimeWarning: overflow encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe')
/Users/chris/anaconda3/envs/python37/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1508: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)


In [23]:
ac.run(n_iters=100001, gamma=0.99)

KeyboardInterrupt: 

In [33]:
sp = gym.spaces.Box(low=np.array([-100]), high=np.array([100]) )
sp.sample()

array([-49.383453], dtype=float32)

In [38]:
fminbound( lambda x : (x-2) ** 2, -10, 10 )

2.0

In [ ]:
E = [env]
A = tf.convert_to_tensor(np.array( [1]))
[ env.step(a) for env, a in zip( E, A ) ]

In [ ]:
gamma = 0.99
n_episodes = 10000

# Get the initial states
batch_states = ac.env.reset()
rewards_mean, rewards_max, rewards_min = [], [], []
entropy_history = []        
curr_scores = np.zeros((len(batch_states),))
scores = []

for i in range(n_episodes):
    batch_actions = ac.choose_action_from_state(batch_states)
    batch_next_states, batch_rewards, batch_done, _ = ac.env.step(batch_actions)

    # Increment the scores, and reward scores for games that have finished
    curr_scores += batch_rewards
    scores.extend(curr_scores[batch_done])
    curr_scores[batch_done] = 0

    # Train the neural network from the states, rewards and transitions
    actor_loss, critic_loss, ent_t = ac.train_step( batch_states, batch_next_states, \
                                  batch_actions, batch_rewards, batch_done, gamma=gamma )
    batch_states = batch_next_states
    entropy_history.append(np.mean(ent_t))

    if np.any(batch_done):
        print( '{}: Finished with score of: {}'.format(i, scores[-1] ) )
        clear_output(True)
        plt.figure(figsize=[14, 6])
        plt.grid()
        plt.plot(scores)
        plt.show()

In [ ]:
import gym
dt = list(gym.envs.registry.all())
kenv = gym.make( 'KellyCoinflip-v0')

In [ ]:
kenv.observation_space[0].low[0]

In [ ]:
gym.envs.classic_control

In [ ]:
np.hstack( [ [0], np.array(3) ])

In [ ]:
from collections import namedtuple
Observation = namedtuple( 'Observation', [ 'portfolio_value', 'weight', 'mu', 'cholesky' ])

In [ ]:
np.tril_indices_from?

In [ ]:
import tensorflow_probability as tfp

In [ ]:
R = np.random.randn(1000,2) 
R = R - R.mean(axis=1, keepdims=True)

In [ ]:
plt.scatter( 0.3 + 0.2 * R[:,0], 0.7 + 0.2 * R[:,1])
plt.xlim( [0,1])
plt.ylim([0, 1] )

In [ ]:
plt.hist(0.9 + 0.2 * R[:,1])

In [ ]:
np.random.dirichlet( np.ones((4,)) )

In [ ]:
rangen = np.random.RandomState()